In [34]:
import glob
import os
import pprint
import traceback

import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import datetime

# Extraction function
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame

    Parameters
    ----------
    path : str
        path to tensorflow log file

    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame({
        "metric": [], 
        "value": [], 
        "step": [],
        "time": []
        })
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            time = list(map(lambda x: datetime.datetime.fromtimestamp(x.wall_time).strftime('%Y-%m-%d %I:%M:%S %p'), event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step, "time": time}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data

In [35]:
def many_logs2pandas(event_paths):
    all_logs = pd.DataFrame()
    for path in event_paths:
        log = tflog2pandas(path)
        if log is not None:
            if all_logs.shape[0] == 0:
                all_logs = log
            else:
                all_logs = all_logs.append(log, ignore_index=True)
    return all_logs

In [36]:
paths = [
    "../runs/shakespeare_big/events.out.tfevents.1735333983.fedora.684784.0",
]

In [37]:
data = many_logs2pandas(paths)

In [38]:
data

,metric,value,step,time
0,Loss/Training,12.534144,0.0,2024-12-27 01:13:03 PM
1,Loss/Training,12.490757,0.0,2024-12-27 01:13:04 PM
2,Loss/Training,10.509276,1.0,2024-12-27 01:13:05 PM
3,Loss/Training,10.539508,1.0,2024-12-27 01:13:06 PM
4,Loss/Training,10.538725,2.0,2024-12-27 01:13:06 PM
...,...,...,...,...
111,Loss/Training,7.446693,55.0,2024-12-27 01:13:20 PM
0,Loss/Test,9.149016,0.0,2024-12-27 01:13:04 PM
1,Loss/Test,8.500660,0.0,2024-12-27 01:13:04 PM
2,Loss/Test,7.138374,32.0,2024-12-27 01:13:14 PM
